In [1]:
import polars as pl
from export_names import get_exports

In [6]:
output_csv = '../game-dump/names-map.csv'

In [3]:
broken_dlls = [
    r"C:\Windows\System32\kernel32.dll",
    r"G:\Games\FA\FA-EMU\Shipping\fmod_event_net.dll",
    r"G:\Games\FA\FA-EMU\Shipping\awesomium.dll",
    r"G:\Games\FA\FA-EMU\Shipping\fmodex.dll",
    r"G:\Games\FA\FA-EMU\Shipping\fmod_event.dll",
    r"G:\Games\FA\FA-EMU\Shipping\umbraoptimizer32.dll",
    # r"G:\Games\FA\FA-EMU\Shipping\msvcp90.dll",
    # r"G:\Games\FA\FA-EMU\Shipping\msvcr90.dll",
    r"C:\Windows\WinSxS\x86_microsoft.vc90.crt_1fc8b3b9a1e18e3b_9.0.30729.9625_none_508ef7e4bcbbe589\msvcp90.dll",
    r"C:\Windows\WinSxS\x86_microsoft.vc90.crt_1fc8b3b9a1e18e3b_9.0.30729.9625_none_508ef7e4bcbbe589\msvcr90.dll"
]

In [4]:
schema = {
    'decorated': pl.String,
    'undecorated': pl.String,
}

df = pl.DataFrame(schema=schema)

for dll in broken_dlls:
    _, m = get_exports(dll)
    for u, d in m.items():
        df = df.vstack(pl.DataFrame({
            'decorated': d[0],
            'undecorated': u,
        }, schema=schema))

df = df.vstack(pl.DataFrame({
    'decorated': [
        "??_D?$basic_ofstream@DU?$char_traits@D@std@@@std@@QAEXXZ",
        "??_D?$basic_ifstream@DU?$char_traits@D@std@@@std@@QAEXXZ"
    ],
    'undecorated': [
        "public: void __thiscall std::basic_ofstream<char, struct std::char_traits<char>>::`vbase dtor'(void)",
        "public: void __thiscall std::basic_ifstream<char, struct std::char_traits<char>>::`vbase dtor'(void)"
    ], 
}))

df

decorated,undecorated
str,str
"""AcquireSRWLockExclusive""","""AcquireSRWLockExclusive"""
"""AcquireSRWLockShared""","""AcquireSRWLockShared"""
"""ActivateActCtx""","""ActivateActCtx"""
"""ActivateActCtxWorker""","""ActivateActCtxWorker"""
"""AddAtomA""","""AddAtomA"""
…,…
"""wprintf_s""","""wprintf_s"""
"""wscanf""","""wscanf"""
"""wscanf_s""","""wscanf_s"""


In [7]:
df.write_csv(output_csv)

# Troubleshooting

In [ ]:
import lief

In [ ]:
path = r"G:\Games\FA\FA-EMU\Shipping\msvcp90.dll"
path = r"C:\Windows\WinSxS\x86_microsoft.vc90.crt_1fc8b3b9a1e18e3b_9.0.30729.9625_none_508ef7e4bcbbe589\msvcp90.dll"

In [ ]:
dll = lief.parse(path)

In [ ]:
for func in dll.exported_functions:
    if 'ostream@DU?$char_traits@D@std@@@std@@QAEXXZ' in func.name:
        print(func.name, func.name == "??_D?$basic_ostream@DU?$char_traits@D@std@@@std@@QAEXXZ")

??_D?$basic_iostream@DU?$char_traits@D@std@@@std@@QAEXXZ False
??_D?$basic_ostream@DU?$char_traits@D@std@@@std@@QAEXXZ True
?_Osfx@?$basic_ostream@DU?$char_traits@D@std@@@std@@QAEXXZ False
?osfx@?$basic_ostream@DU?$char_traits@D@std@@@std@@QAEXXZ False


In [ ]:
# Адрес=57148D56
# Тип=Экспорт
# Порядковый номер=1030
# Символ=??_D?$basic_ofstream@DU?$char_traits@D@std@@@std@@QAEXXZ
# Символ (undecorated)=public: void __thiscall std::basic_ofstream<char, struct std::char_traits<char>>::`vbase dtor'(void)


In [ ]:
# Адрес=57148CF3
# Тип=Экспорт
# Порядковый номер=1018
# Символ=??_D?$basic_ifstream@DU?$char_traits@D@std@@@std@@QAEXXZ
# Символ (undecorated)=public: void __thiscall std::basic_ifstream<char, struct std::char_traits<char>>::`vbase dtor'(void)
